In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Buscar los comentarios
comment_elements = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprimir comentarios
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")


Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


In [2]:
with open('pagina_html.txt', 'w', encoding='utf-8') as file:
    file.write(soup.prettify())

In [9]:
# Ejemplo de uso de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Ruta de tu ChromeDriver
driver_path = "D:\\Proyecto Web Scrapping\\chromedriver.exe"  # Nota: usa doble backslash "\\" en Windows
service = Service(driver_path)

# Configurar ChromeDriver con el servicio
driver = webdriver.Chrome(service=service)
driver.get("https://www.google.com")
print("Título de la página:", driver.title)
driver.quit()

Título de la página: Google


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

from selenium.webdriver.chrome.options import Options

# Configuración para simular un navegador real
chrome_options = Options()

# Evitar ser detectado por automatización
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")

# Agregar un user-agent personalizado
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)

# Inicializar el driver con las opciones configuradas
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL de la página
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320'
driver.get(url)

# Espera inicial para cargar la página
time.sleep(5)

# Haz clic en el botón "Mostrar todas las opiniones" para abrir el modal
try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.show-more-click[data-testid="see-more"]')
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more_button)  # Haz clic en el botón
    time.sleep(5)  # Ajustar el tiempo si es necesario
    print("Botón 'Mostrar todas las opiniones' clicado.")
except NoSuchElementException:
    print("No se encontró el botón para abrir los comentarios.")
    driver.quit()
    exit()

# Espera a que el modal de comentarios esté completamente cargado
try:
    modal_container = WebDriverWait(driver, 20).until(  # Incrementa el tiempo de espera
        EC.presence_of_element_located((By.CSS_SELECTOR, '.andes-modal__content'))
    )
    print("Modal de comentarios encontrado.")
except TimeoutException:
    print("El modal de comentarios no se cargó a tiempo. Guardando el HTML completo para depuración.")
    # Guardar el HTML completo para inspección
    with open("page_debug.html", "w", encoding="utf-8") as file:
        file.write(driver.page_source)

    driver.quit()
    exit()

# Forzar el scroll dentro del modal
scroll_pause_time = 2  # Tiempo de espera entre cada scroll

try:
    last_height = driver.execute_script("return arguments[0].scrollHeight;", modal_container)
    while True:
        # Ejecuta el scroll hacia el final
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", modal_container)
        time.sleep(scroll_pause_time)
        
        # Verifica si la altura ya no cambia
        new_height = driver.execute_script("return arguments[0].scrollHeight;", modal_container)
        if new_height == last_height:
            print("Se alcanzó el final del modal.")
            break
        last_height = new_height
except Exception as e:
    print(f"Error durante el scroll: {e}")
    driver.quit()
    exit()

# Extrae los comentarios con BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
comment_elements = soup.select('p.ui-review-capability-comments__comment__content')  # Selector para comentarios en el modal
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprime los comentarios extraídos
if comments:
    for idx, comment in enumerate(comments, 1):
        print(f"Comentario {idx}: {comment}")
else:
    print("No se encontraron comentarios con el selector actual.")

# Guardar los comentarios en un archivo para inspección
with open("comentarios_extraidos.txt", "w", encoding="utf-8") as file:
    for comment in comments:
        file.write(comment + "\n")

# Cierra el navegador
driver.quit()


Botón 'Mostrar todas las opiniones' clicado.
Modal de comentarios encontrado.
Se alcanzó el final del modal.
Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


Webb Scrapping Falabella

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# Ruta del driver
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'  # Cambia esta ruta si es necesario
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL del producto
url = "https://www.falabella.com.co/falabella-co/product/129586393/CELULAR-SAMSUNG-GALAXY-A55-5G-256GB-8-RAM-AZUL-NAVY/129586394"
driver.get(url)

# Esperar a que se cargue el contenedor de comentarios
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='user-reviews-container']")))

# Cargar todos los comentarios
while True:
    try:
        # Localizar y hacer clic en el botón "Ver más comentarios"
        ver_mas_btn = driver.find_element(By.ID, "more-comments-button")
        driver.execute_script("arguments[0].scrollIntoView();", ver_mas_btn)  # Asegurar que el botón está visible
        ver_mas_btn.click()  # Hacer clic
        time.sleep(2)  # Esperar que se carguen los nuevos comentarios
    except Exception as e:
        print("No hay más comentarios para cargar.")
        break

# Extraer el HTML del contenedor completo
html_content = driver.find_element(By.CSS_SELECTOR, "div[data-testid='user-reviews-container']").get_attribute('innerHTML')

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
comentarios = soup.select("div._review-text_16yc3_2")  # Seleccionar todos los textos de comentarios

# Extraer los textos de los comentarios
data = [comentario.get_text(strip=True) for comentario in comentarios]

# Guardar los datos en un archivo CSV
filename = "comentarios_falabella.csv"
with open(filename, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["comentario"])  # Cabecera
    for texto in data:
        writer.writerow([texto])

print(f"Se han guardado {len(data)} comentarios en {filename}")

# Cerrar el navegador
driver.quit()


No hay más comentarios para cargar.
Se han guardado 2165 comentarios en comentarios_falabella.csv


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

%pip install pandas
import pandas as pd

# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# Ruta del driver
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'  # Cambia esta ruta si es necesario
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Lista de productos (enlaces y modelos)
productos = [
    # Xiaomi
    ("https://www.falabella.com.co/falabella-co/product/127913189/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Negro/127913190", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp"),
    ("https://www.falabella.com.co/falabella-co/product/127913775/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Amarillo/127913776", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp"),
    ("https://www.falabella.com.co/falabella-co/product/127916553/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5G-512Gb-12Ram-200Mp-Negro/127916554", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp"),
    ("https://www.falabella.com.co/falabella-co/product/128104744/Celular-Xiaomi-Redmi-Note-13-PRO-PLUS-5G-512GB-12GB-RAM-Negro/128104745", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp"),
    ("https://www.falabella.com.co/falabella-co/product/130055074/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5g-512gb-12ram-PURPURA/130055075", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp"),
    ("https://www.falabella.com.co/falabella-co/product/127687136/CELULAR-XIAOMI-REDMI-13C-256GB-8GB-RAM-NEGRO/127687137", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/128261646/Celular-Xiaomi-Redmi-13C-256GB-8RAM-Negro/128261647", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/128128076/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Negro/128128077", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/128807587/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Purpura/128807589", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/prod13530339/Celular-Xiaomi-Redmi-Note-13-Pro-256GB-8GB-Camara-Posterior-200MP+8MP+2MP-Camara-Frontal-16MP-Pantalla-6.67-Pulgadas/72853341", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/128128195/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Forest-Green/128128196", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/128931528/Celular-Xiaomi-Redmi-Note-13-8GB-256GB-Negro/128931529", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/127891485/Celular-Xiaomi-Redmi-Note-13-256GB-8GB-RAM-Negro/127891486", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB"),
    
    # Samsung
    ("https://www.falabella.com.co/falabella-co/product/129586393/CELULAR-SAMSUNG-GALAXY-A55-5G-256GB-8-RAM-AZUL-NAVY/129586394", "Samsung", "SAMSUNG GALAXY A55"),
    ("https://www.falabella.com.co/falabella-co/product/prod13530496/Celular-Samsung-A15-256GB-8GB-RAM-Camara-Posterior-50-MP-Camara-Frontal-12MP-Pantalla-6.5-Pulgadas-+-MediaTek-MT6789V-CD-/72854633", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/132236410/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL-NEGRO/132236411", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/128004219/CELULAR-SAMSUNG-GALAXY-A15-256GB-8-RAM-NEGRO-(Blue-Black)/128004220", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/132236494/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL/132236495", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/128906319/Celular-Samsung-Galaxy-A25-5G-256GB-8RAM+-Cargador/128906321", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM"),
    ("https://www.falabella.com.co/falabella-co/product/131931272/Celular-SAMSUNG-GALAXY-A25-256GB-+-8-RAM-AZUL-CLARO/131931273", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM"),
    ("https://www.falabella.com.co/falabella-co/product/72899217/Celular-Samsung-Galaxy-A35-5G-256GB-8GB-RAM-camara-posterior-50-MP-camara-frontal-13MP-pantalla-6.6-Pulgadas-+-Exynos-1380/72899217", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/133744591/Celular-Samsung-Galaxy-A35-256Gb-5g-Lila/133744592", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/128668464/Celular-Samsung-Galaxy-A05S-128Gb-Negro/128668465", "Samsung", "Samsung Galaxy A05S 128Gb"),
    ("https://www.falabella.com.co/falabella-co/product/128035979/Samsung-S24-Ultra-256GB-12GB-Violet/128035980", "Samsung", "Samsung S24 Ultra 256GB 12GB"),
    ("https://www.falabella.com.co/falabella-co/product/128853893/Celular-Samsung-Galaxy-S24-Ultra-De-256GB12GB-RAM-Gris-5G/128853894", "Samsung", "Samsung S24 Ultra 256GB 12GB"),
    ("https://www.falabella.com.co/falabella-co/product/129574090/Samsung-Galaxy-S24-Ultra-512GB-12GB-Ram-5G-Violeta/129574091", "Samsung", "Samsung Galaxy S24 Ultra 512GB 12GB"),

    # Apple
    ("https://www.falabella.com.co/falabella-co/product/prod12000048/iPhone-13-128-GB-5G-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic/19604508", "Apple", "iPhone 13 128 GB 5G 4GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/prod13430667/iPhone-15-de-128GB-5G-6GB-RAM-Pantalla-6.1-pulgadas-Chip-A16-Bionic-Carga-Tipo-C-Dynamic-Island-Camara-48MP/72752105", "Apple", "iPhone 15 128GB 5G 6GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/prod13070047/iPhone-14-128GB-5G-6GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic-/60999138", "Apple", "iPhone 14 128GB 5G 6GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/prod10900027/iPhone-11-128-GB-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A13-Bionic-Camara-12-MP/9722720", "Apple", "iPhone 11 128 GB 4GB RAM"),
]



# Lista para almacenar los datos
data = []

# Iterar sobre cada producto
for url, marca, modelo in productos:
    print(f"Procesando: {modelo} ({marca})")

    # Cargar la página del producto
    driver.get(url)

    # Esperar a que se cargue el contenedor de comentarios
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='user-reviews-container']")))

        # Cargar todos los comentarios
        while True:
            try:
                ver_mas_btn = driver.find_element(By.ID, "more-comments-button")
                driver.execute_script("arguments[0].scrollIntoView();", ver_mas_btn)  # Asegurar que el botón está visible
                ver_mas_btn.click()
                time.sleep(3)  # Esperar que se carguen los nuevos comentarios
            except Exception:
                print("No hay más comentarios para cargar.")
                break

        # Extraer el HTML del contenedor de comentarios
        html_content = driver.find_element(By.CSS_SELECTOR, "div[data-testid='user-reviews-container']").get_attribute('innerHTML')
        soup = BeautifulSoup(html_content, "html.parser")

        # Extraer los comentarios
        comentarios = soup.select("div._review-text_16yc3_2")
        for comentario in comentarios:
            texto = comentario.get_text(strip=True)
            data.append({"comentario": texto, "marca": marca, "modelo": modelo})

    except Exception as e:
        print(f"Error procesando {modelo}: {e}")

# Crear un DataFrame con los resultados
df = pd.DataFrame(data)

# Exportar a CSV
df.to_csv("comentarios_productos.csv", index=False, encoding="utf-8")

print(f"Se procesaron {len(data)} comentarios en total. Archivo guardado como 'comentarios_productos.csv'.")

# Cerrar el navegador
driver.quit()

#Pasar comentarios_productos.csv a dataframe
import pandas as pd
df = pd.read_csv('comentarios_productos.csv')







[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/11.6 MB ? eta -:--:--
     -                                        0.3/11.6 MB 6.5 MB/s eta 0:00:02
     ---                                      1.0/11.6 MB 13.3 MB/s eta 0:00:01
     -----                                    1.5/11.6 MB 12.0 MB/s eta 0:00:01
     ---------                                2.7/11.6 MB 15.5 MB/s eta 0:00:01
     ------------                             3.8/11.6 MB 17.2 MB/s eta 0:00:01
     ---------------                          4.6/11.6 MB 17.2 MB/s eta 0:00:01
     -------------------                      5.6/11.6 MB 18.0 MB/s eta 0:00:01
     ----------------------                   6.4/11.6 MB 17.8 MB/s eta 0:00:01
     -------------------------                7.3/11.6 MB 17.9 MB/s eta 0:00:01
     ---------------------------              8.1/11.6 MB 17.9 MB/s eta 0:00:01
     -----------------------------            8.5/11.6 MB 18.2 MB/s eta 0:00:01
     ---------------------------------       10.0

In [3]:
import pandas as pd
df = pd.read_csv('comentarios_productos.csv')
df = pd.read_csv('comentarios_productos.csv')
df.head()
#mostrar valores unicos de la columna modelo
df['modelo'].unique()

array(['Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp',
       'Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp',
       'XIAOMI REDMI 13C 256GB 8GB RAM',
       'Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram',
       'Xiaomi Redmi Note 13 8GB 256GB', 'SAMSUNG GALAXY A55',
       'Samsung A15 256GB', 'Samsung Galaxy A25 5G 256GB 8RAM',
       'Samsung Galaxy A35 5G 256GB 8GB RAM', 'Samsung Galaxy A05S 128Gb',
       'Samsung S24 Ultra 256GB 12GB',
       'Samsung Galaxy S24 Ultra 512GB 12GB',
       'iPhone 13 128 GB 5G 4GB RAM', 'iPhone 15 128GB 5G 6GB RAM',
       'iPhone 14 128GB 5G 6GB RAM', 'iPhone 11 128 GB 4GB RAM'],
      dtype=object)

In [7]:
#Contar cuantos comentarios hay por producto y por marca en comentarios_productos.csv
df.groupby(['marca', 'modelo']).size()



marca    modelo                                            
Apple    iPhone 11 128 GB 4GB RAM                              233
         iPhone 13 128 GB 5G 4GB RAM                            82
         iPhone 14 128GB 5G 6GB RAM                             93
         iPhone 15 128GB 5G 6GB RAM                             53
Samsung  SAMSUNG GALAXY A55                                     18
         Samsung A15 256GB                                     115
         Samsung Galaxy A05S 128Gb                              11
         Samsung Galaxy A25 5G 256GB 8RAM                       90
         Samsung Galaxy A35 5G 256GB 8GB RAM                   128
         Samsung Galaxy S24 Ultra 512GB 12GB                    35
         Samsung S24 Ultra 256GB 12GB                           69
Xiaomi   XIAOMI REDMI 13C 256GB 8GB RAM                         46
         Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp                 98
         Xiaomi Redmi Note 13 8GB 256GB                         66
  

In [8]:
#Contar cuantos comentarios hay por por marca en comentarios_productos.csv
df.groupby(['marca']).size()

marca
Apple      461
Samsung    466
Xiaomi     451
dtype: int64